In [219]:
ES_TRAIN_DATA_FILE = "./ES/train"
ES_TEST_DATA_FILE = "./ES/dev.in"
ES_ACTUAL_DATA_FILE = "./ES/dev.out"
ES_PART1_PREDICTED_DATA_FILE = "./ES/dev.p1.out"
ES_PART2_PREDICTED_DATA_FILE = "./ES/dev.p2.out"
ES_PART3_PREDICTED_DATA_FILE = "./ES/dev.p3.out"
ES_PART4_PREDICTED_DATA_FILE = "./ES/dev.p4.out"

RU_TRAIN_DATA_FILE = "./RU/train"
RU_TEST_DATA_FILE = "./RU/dev.in"
RU_ACTUAL_DATA_FILE = "./RU/dev.out"
RU_PART1_PREDICTED_DATA_FILE = "./RU/dev.p1.out"
RU_PART2_PREDICTED_DATA_FILE = "./RU/dev.p2.out"
RU_PART3_PREDICTED_DATA_FILE = "./RU/dev.p3.out"
RU_PART4_PREDICTED_DATA_FILE = "./RU/dev.p4.out"

UNK_WORD = "#UNK#"
START_TAG = "START"
STOP_TAG = "STOP"

In [220]:
def _preprocess_training_file(file_path):
    with open(file_path, encoding="utf8") as f:
        data = f.read().splitlines()
        data[:] = [x for x in data if x]

    output = []
    for i in data:
        i = i.split(" ")
        if(len(i) > 2):
            i = [" ".join(i[0:len(i)-1]), i[len(i)-1]]
            output.append(i)
        else:
            output.append(i)
        
    return output

def _preprocess_testing_file(path):
    with open(path, encoding="utf8") as f:
        data = f.read().splitlines()

    output = []
    for word in data:
        # if word: # leave the newlines or not?
        output.append(word)

    return output

In [221]:
def get_emission_using_MLE(training, k = 1):
    tags = {}
    tags_to_word = {}
    emission = {}
    for data in training:
        word, tag = data[0], data[1]
        if tag in tags:
            tags[tag] += 1
        else:
            tags[tag] = 1
        
        tag_to_word = (tag, word)
        if tag_to_word in tags_to_word:
            tags_to_word[tag_to_word] += 1
        else:
            tags_to_word[tag_to_word] = 1

    for key in tags_to_word.keys():
        emission[key] = tags_to_word[key] / (tags[key[0]] + k)
    for key in tags.keys():
        transition = (key, UNK_WORD)
        emission[transition] = k / (tags[key] + k)
    # print(emission)
    return emission

In [222]:
def get_most_probable_tag(emission):
    highest_prob = {}
    output = {}
    for key, prob in emission.items():
        tag, word = key[0], key[1]
        if word not in highest_prob:
            highest_prob[word] = prob
            output[word] = tag
        else:
            if prob > highest_prob[word]:
                highest_prob[word] = prob
                output[word] = tag
    # print(output)
    return output

In [225]:
# Part 1 code for ES Dataset
ES_train_data = _preprocess_training_file(ES_TRAIN_DATA_FILE)
ES_test_data = _preprocess_testing_file(ES_TEST_DATA_FILE)

ES_emission_parameters = get_emission_using_MLE(ES_train_data)
ES_most_probable_tag = get_most_probable_tag(ES_emission_parameters)

# Part 1 code for RU Dataset
RU_train_data = _preprocess_training_file(RU_TRAIN_DATA_FILE)
RU_test_data = _preprocess_testing_file(RU_TEST_DATA_FILE)

RU_emission_parameters = get_emission_using_MLE(RU_train_data)
RU_most_probable_tag = get_most_probable_tag(RU_emission_parameters)

In [227]:
import itertools
def _preprocess_training_file(training_file):
    tags = []
    tags_with_start_stop = []
    words = []

    with open(training_file,"r",encoding="utf8") as f:
        document = f.read().rstrip()
        lines = document.split("\n\n")

        for line in lines:
            tags_list = []
            tags_with_start_stop_list = []
            words_list = []

            for word_tag in line.split("\n"):
                i = word_tag.split(" ")

                if len(i) > 2:
                    i = [" ".join(i[0:len(i)-1]), i[len(i)-1]]

                word, tag = i[0], i[1]
                words_list.append(word)
                tags_list.append(tag)

            tags.append(tags_list)
            tags_with_start_stop_list = [START_TAG] + tags_list + [STOP_TAG]
            tags_with_start_stop.append(tags_with_start_stop_list)
            words.append(words_list)
    
    # print(f'tags: {tags[0:3]}')
    # print(f'tags_with_start_stop: {tags_with_start_stop[0:3]}')
    # print(f'words: {words[0:3]}\n')
    return tags, tags_with_start_stop, words

def _preprocess_test_file(testing_file):
    test_words = []

    with open(testing_file,"r",encoding="utf8") as f:
        document = f.read().rstrip()
        lines = document.split("\n\n")

        for line in lines:
            word_list = []
            for word in line.split("\n"):
                word_list.append(word)
            test_words.append(word_list)

    return test_words

In [228]:
def get_transition_pairs(tags):
    transition_pair_count = {}

    for tag in tags:
        for tag1, tag2 in zip(tag[:-1], tag[1:]):
            transition_pair = (tag1, tag2)
            if transition_pair in transition_pair_count:
                transition_pair_count[transition_pair] += 1
            else:
                transition_pair_count[transition_pair] = 1

    return transition_pair_count

In [ ]:
def get_unique_elements(x):
    return list(set(list(itertools.chain.from_iterable(x))))

In [229]:
def count_y(tag, tags):
    tags_flattened = list(itertools.chain.from_iterable(tags))
    return tags_flattened.count(tag)

In [230]:
def get_transition_using_MLE(unique_tags, transition_pair_count,tags_with_start_stop):
    unique_tags = [START_TAG] + unique_tags + [STOP_TAG]
    transition = {}
    for u in unique_tags[:-1]: # omit STOP
        transition_row = {}
        for v in unique_tags[1:]: # omit START
            transition_row[v] = 0.0
        transition[u] = transition_row

    # print(f'transition_params_init: {transition}\n')
    # populate transition parameters with counts
    for u, v in transition_pair_count:
        transition[u][v] += transition_pair_count[(u, v)]
    
    # print(f'transition_params_with_count: {transition}\n')

    # divide transition_count by count_yi, to get probability
    for u, transition_row in transition.items():
        count_yi = count_y(u, tags_with_start_stop)
        # words in training set
        for v, transition_count in transition_row.items():
            if count_yi == 0:
                transition[u][v] = 0.0
            else:
                transition[u][v] = transition_count / count_yi

    # print(f'transition_params_final: {transition}\n')
    return transition

In [231]:
ES_tags, ES_tags_with_start_stop, ES_train_words = _preprocess_training_file(ES_TRAIN_DATA_FILE)
ES_test_words = _preprocess_test_file(ES_TEST_DATA_FILE)
ES_unique_words = get_unique_elements(ES_train_words)
ES_unique_tags = get_unique_elements(ES_tags)

ES_transition_pair_count = get_transition_pairs(ES_tags_with_start_stop)
ES_transition_parameters = get_transition_using_MLE(ES_unique_tags, ES_transition_pair_count, ES_tags_with_start_stop)

RU_tags, RU_tags_with_start_stop, RU_train_words = _preprocess_training_file(RU_TRAIN_DATA_FILE)
RU_test_words = _preprocess_test_file(RU_TEST_DATA_FILE)
RU_unique_words = get_unique_elements(RU_train_words)
RU_unique_tags = get_unique_elements(RU_tags)

RU_transition_pair_count = get_transition_pairs(RU_tags_with_start_stop)
RU_transition_parameters = get_transition_using_MLE(RU_unique_tags, RU_transition_pair_count, RU_tags_with_start_stop)


# Part 4 - Viterbi Algorithm
Improved sentiment analysis system
## Add-one smoothing / Laplace smoothing 

In [232]:
# TODO: Song Gee to write algorithm for Viterbi
import math
import sys

word_output_list = []  # list of tuple(word, predicted_tag) for writing to file
viterbi_values = {}  # key: (n, tag)  value: float 

def generate_viterbi_values(n, current_tag, word_list, words_unique, tags_unique, emission_params, transmission_params):
    global viterbi_values
    current_max_viterbi_value = -sys.float_info.max  # Smallest negative float

    if n == 0:
        return

    if n == 1:
        try:
            if word_list[n-1] in words_unique:
                try:
                    current_max_viterbi_value = math.log(emission_params[(current_tag, word_list[n-1])] * transmission_params[START_TAG][current_tag])
                except KeyError:
                    current_max_viterbi_value = -sys.float_info.max
            else:
                current_max_viterbi_value = math.log(emission_params[(current_tag, UNK_WORD)] * transmission_params[START_TAG][current_tag])
        except ValueError:
            current_max_viterbi_value = -sys.float_info.max
        
        viterbi_values[(n, current_tag)] = current_max_viterbi_value
        return
        

    # Recursive call to generate viterbi_values for (n-1, tag)
    for tag in tags_unique:
        if (n-1, tag) not in viterbi_values:
            generate_viterbi_values(n-1, tag, word_list, words_unique, tags_unique, emission_params, transmission_params)

    # Use viterbi values from n-1 to generate current viterbi value
    for tag in tags_unique:
        # Here, we use a try-except block because our emission parameters only contain emissions which appeared in our datasets
        # Thus, any unobserved emission will throw a KeyError, however it's value should be -inf, so we just catch the Error and proceed to the next tag
        # If transmission_params gives 0, math.log will throw a valueError, thus we catch it and skip the current tag since 0 means we should never consider it
        try:
            if word_list[n-1] in words_unique:
                try:
                    value = viterbi_values[(n-1, tag)] + math.log(emission_params[(current_tag, word_list[n-1])] * transmission_params[tag][current_tag])
                except KeyError:
                    continue
            else:
                value = viterbi_values[(n-1, tag)] + math.log(emission_params[(current_tag, UNK_WORD)] * transmission_params[tag][current_tag])
        except ValueError:
            continue
        
        # print(f'n: {n}, current_tag: {current_tag}, tag: {tag}, value: {value}')
        current_max_viterbi_value = max(current_max_viterbi_value, value)
    
    viterbi_values[(n, current_tag)] = current_max_viterbi_value


# function to kickstart viterbi recursive chain, and add (n+1, STOP) to veterbi_values
def start_viterbi(word_list, words_unique, tags_unique, emission_params, transmission_params):
    global viterbi_values
    max_final_viterbi_value = -sys.float_info.max

    n = len(word_list)

    # Recursive call to generate viterbi_values for (n, tag)
    for tag in tags_unique:
        generate_viterbi_values(n, tag, word_list, words_unique, tags_unique, emission_params, transmission_params)

    # Use viterbi values from n to generate viterbi value for (n+1, STOP)
    for tag in tags_unique:
        try:
            value = viterbi_values[(n, tag)] + math.log(transmission_params[tag][STOP_TAG])
        except ValueError:
            continue
        max_final_viterbi_value = max(max_final_viterbi_value, value)

    viterbi_values[(n+1, STOP_TAG)] = max_final_viterbi_value


def generate_predictions_viterbi(word_list, tags_unique, transmission_params):
    global viterbi_values

    n = len(word_list)

    generated_tag_list = ['' for i in range(n)]

    # Compute tag for n
    current_best_tag = 'O'
    current_best_tag_value = -sys.float_info.max

    for tag in tags_unique:
        try:
            value = viterbi_values[(n, tag)] + math.log(transmission_params[tag][STOP_TAG])
        except ValueError:
            continue
        if value > current_best_tag_value:
            current_best_tag = tag
            current_best_tag_value = value

    generated_tag_list[n-1] = current_best_tag

    # Generate predictions starting from n-1 going down to 1
    for i in range(n-1, 0, -1):
        current_best_tag = 'O'
        current_best_tag_value = -sys.float_info.max

        for tag in tags_unique:
            try:
                value = viterbi_values[(i, tag)] + math.log(transmission_params[tag][generated_tag_list[i]])
            except ValueError:
                continue
            if value > current_best_tag_value:
                current_best_tag = tag
                current_best_tag_value = value

        generated_tag_list[i-1] = current_best_tag
    return generated_tag_list


def write_to_predicted_file_part2(predicted_file, words_list, tags_list):
    assert len(words_list) == len(tags_list)

    with open(predicted_file, "w", encoding="utf8") as f:
        for words, tags in zip(words_list, tags_list):  # Unpack all sentences and list of tags
            assert len(words) == len(tags)
            for word, tag in zip (words, tags):  # Unpack all words and tags
                f.write(f"{word} {tag}\n")
            f.write("\n")


In [217]:
# Run and output Viterbi for ES
ES_predicted_tags_list = []
for word in ES_test_words:
    viterbi_values = {}
    start_viterbi(word, ES_unique_words, ES_unique_tags, ES_emission_parameters, ES_transition_parameters)
    ES_generated_tag_list = generate_predictions_viterbi(word, ES_unique_tags, ES_transition_parameters)
    ES_predicted_tags_list.append(ES_generated_tag_list)

write_to_predicted_file_part2(ES_PART4_PREDICTED_DATA_FILE, ES_test_words, ES_predicted_tags_list)


# Run and output Viterbi for RU
RU_predicted_tags_list = []
for word in RU_test_words:
    viterbi_values = {}
    start_viterbi(word, RU_unique_words, RU_unique_tags, RU_emission_parameters, RU_transition_parameters)
    RU_generated_tag_list = generate_predictions_viterbi(word, RU_unique_tags, RU_transition_parameters)
    RU_predicted_tags_list.append(RU_generated_tag_list)

write_to_predicted_file_part2(RU_PART4_PREDICTED_DATA_FILE, RU_test_words, RU_predicted_tags_list)


In [218]:
from EvalScript.evalResult import get_observed, get_predicted, compare_observed_to_predicted

def evaluateScores(actual_file, predicted_file):
    with open(predicted_file, encoding="utf8") as f:
        predicted = f.read().splitlines()

    with open(actual_file, encoding="utf8") as f:
        actual = f.read().splitlines()

    compare_observed_to_predicted(get_observed(actual), get_predicted(predicted))

evaluateScores(ES_ACTUAL_DATA_FILE, ES_PART4_PREDICTED_DATA_FILE)
evaluateScores(RU_ACTUAL_DATA_FILE, RU_PART4_PREDICTED_DATA_FILE)


#Entity in gold data: 255
#Entity in prediction: 576

#Correct Entity : 131
Entity  precision: 0.2274
Entity  recall: 0.5137
Entity  F: 0.3153

#Correct Sentiment : 104
Sentiment  precision: 0.1806
Sentiment  recall: 0.4078
Sentiment  F: 0.2503

#Entity in gold data: 461
#Entity in prediction: 552

#Correct Entity : 222
Entity  precision: 0.4022
Entity  recall: 0.4816
Entity  F: 0.4383

#Correct Sentiment : 145
Sentiment  precision: 0.2627
Sentiment  recall: 0.3145
Sentiment  F: 0.2863
